<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/24_06_2024_Combined_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import csv
import re
import time
from datetime import datetime, timedelta
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pytz

def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {'serviceTypes': 'Regular'}

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        route_sections = data.get('routeSections', [])

        found_direction = False

        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')

                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")

                # Fetch and print sequence of stop points
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)

                if all_stop_details:
                    # Write to CSV file
                    write_to_csv(all_stop_details)

                    # Display data from CSV file
                    display_data_from_csv()

        if not found_direction:
            print(f"No route found for direction '{direction}'.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        line_strings = data.get('lineStrings', [])

        if line_strings:
            printed_stop_ids = set()  # To track printed stop IDs

            for line_string in line_strings:
                # Parsing the line string to extract latitudes and longitudes
                coordinates = eval(line_string)  # Evaluating string to list
                for lon, lat in coordinates[0]:
                    # Fetching and appending stop details using the lat and lon
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)

        else:
            print("No stop points found for this route.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")

    return all_stop_details

def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"

    try:
        for attempt in range(retries):
            response = requests.get(url)

            if response.status_code == 429:
                time.sleep(2**attempt)  # Exponential backoff
                continue

            response.raise_for_status()  # Raise exception for bad status codes

            data = response.json()
            stop_points = data.get('stopPoints', [])

            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')

                    # Append to all_stop_details only if stop ID has not been printed before
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')

                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })

                        # Add stop ID to printed set
                        printed_stop_ids.add(stop_id)

            else:
                # If no stop points found, continue to next attempt
                continue

            break  # Exit loop if successful

        else:
            # If maximum retries exceeded, handle quietly (you can add logging or alternative handling here)
            pass

    except requests.exceptions.RequestException as e:
        # Handle other request exceptions (you can add logging or alternative handling here)
        pass

def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']

    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()

        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)

    print(f"Data written to 'stop_details.csv' successfully.")

def display_data_from_csv():
    print("Data from CSV:")

    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)

        # Print headers
        headers_printed = False

        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

def convert_to_bst(dt):
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')  # BST timezone for London

    utc_dt = utc_tz.localize(dt)
    bst_dt = utc_dt.astimezone(bst_tz)

    return bst_dt.strftime('%H:%M:%S')

def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}

        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors

        data = response.json()

        if len(data) == 0:
            print("No arrival predictions found.")
            return None, None

        station_name = data[0]['stationName']

        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)  # Convert GMT to BST
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')  # Format to HH:MM
            })

        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))

        return df, station_name

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None  # Return None if there's an error

def create_dashboard(df, station_name):
    current_time_utc = datetime.utcnow()
    current_time_bst = convert_to_bst(current_time_utc)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=[current_time_bst],
        y=[0],
        mode='markers+text',
        marker=dict(size=40, symbol='square', color='black'),
        name='Stop Point',
        text=f"{current_time_bst}",  # Display only the current time below the square
        textposition='bottom center',
        hoverinfo='text'
    ))

    fig.add_annotation(
        x=current_time_bst,
        y=0,
        text=f"{station_name}<br>{stop_point_id}",
        showarrow=False,
        font=dict(size=14, color="black"),
        yshift=40  # Adjust the yshift to move the label above the square
    )

    y_gap = 0.01  # 1 cm gap in graph units
    for i, row in df.iterrows():
        y_pos = 0 + (i + 1) * y_gap

        fig.add_trace(go.Scatter(
            x=[row['Expected Arrival (BST)']],
            y=[y_pos],
            mode='markers+text',
            marker=dict(size=20, symbol='arrow-bar-down', color='blue'),
            name='Bus Arrival',
            text=f"Bus ID: {row['Vehicle ID']}<br>Arrival Time: {row['Expected Arrival (BST)']}",
            textposition='top center',
            hoverinfo='text'
        ))

    fig.update_layout(
        title=f"Expected Bus Arrivals at {station_name}",
        xaxis_title="Time",
        yaxis_title="",
        yaxis=dict(showticklabels=False),
        showlegend=False,
        height=400,
        margin=dict(l=0, r=0, t=40, b=0)
    )

    fig.show()

# Sample Inputs
line_id = input("Enter the Line ID (e.g., 205, 25): ")
direction = input("Enter the direction (inbound or outbound): ")
buses_per_hour = int(input("Enter the number of buses per hour: "))

fetch_route_details(line_id, direction)

df_stops = pd.read_csv('stop_details.csv')

print(df_stops)

selected_stop_input = input("Enter the row number, stop ID, or stop name of the stop you're interested in: ")

if selected_stop_input.isdigit():
    selected_stop = df_stops.loc[df_stops['Sl. No.'] == int(selected_stop_input)]
else:
    selected_stop = df_stops.loc[
        (df_stops['Stop ID'].str.contains(selected_stop_input, case=False)) |
        (df_stops['Stop Name'].str.contains(selected_stop_input, case=False))
    ]

if selected_stop.empty:
    print("No matching stop found.")
else:
    stop_point_id = selected_stop.iloc[0]['Stop ID']
    stop_point_name = selected_stop.iloc[0]['Stop Name']

    df_predictions, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)

    if df_predictions is not None:
        print(df_predictions)
        create_dashboard(df_predictions, station_name)


Enter the Line ID (e.g., 205, 25): 214
Enter the direction (inbound or outbound): inbound
Enter the number of buses per hour: 6
Direction: inbound
Origination Name: Finsbury Square
Destination Name: Highgate School
Originator: 490015209C
Destination: 490008160N
---
Data written to 'stop_details.csv' successfully.
Data from CSV:
1      490015209K Finsbury Square / Moorgate
2      490015209C Finsbury Square
3      490006850J Finsbury Square / Moorgate
4      490006850G Finsbury Square
5      490006850H Finsbury Square
6      490003831E Finsbury Street
7      490015209D Finsbury Square
8      490003831W Finsbury Street
9      490000149B Moorgate Station
10     490000149M Moorgate
11     490006850F Finsbury Square
12     490006612E Epworth Street
13     490000169ZA Old Street Station
14     490015195N Old Street
15     490015195M City Road / Leonard Street
16     490000169K Shoreditch Fire Station
17     490015194F Old Street Station
18     490000169L Old Street Station
19     490010109E O

In [ ]:
import requests
import csv
import re
import time
from datetime import datetime, timedelta
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pytz

def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {'serviceTypes': 'Regular'}

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        route_sections = data.get('routeSections', [])

        found_direction = False

        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')

                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")

                # Fetch and print sequence of stop points
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)

                if all_stop_details:
                    # Write to CSV file
                    write_to_csv(all_stop_details)

                    # Display data from CSV file
                    display_data_from_csv()

        if not found_direction:
            print(f"No route found for direction '{direction}'.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        line_strings = data.get('lineStrings', [])

        if line_strings:
            printed_stop_ids = set()  # To track printed stop IDs

            for line_string in line_strings:
                # Parsing the line string to extract latitudes and longitudes
                coordinates = eval(line_string)  # Evaluating string to list
                for lon, lat in coordinates[0]:
                    # Fetching and appending stop details using the lat and lon
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)

        else:
            print("No stop points found for this route.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")

    return all_stop_details

def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"

    try:
        for attempt in range(retries):
            response = requests.get(url)

            if response.status_code == 429:
                time.sleep(2**attempt)  # Exponential backoff
                continue

            response.raise_for_status()  # Raise exception for bad status codes

            data = response.json()
            stop_points = data.get('stopPoints', [])

            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')

                    # Append to all_stop_details only if stop ID has not been printed before
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')

                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })

                        # Add stop ID to printed set
                        printed_stop_ids.add(stop_id)

            else:
                # If no stop points found, continue to next attempt
                continue

            break  # Exit loop if successful

        else:
            # If maximum retries exceeded, handle quietly (you can add logging or alternative handling here)
            pass

    except requests.exceptions.RequestException as e:
        # Handle other request exceptions (you can add logging or alternative handling here)
        pass

def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']

    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()

        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)

    print(f"Data written to 'stop_details.csv' successfully.")

def display_data_from_csv():
    print("Data from CSV:")

    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)

        # Print headers
        headers_printed = False

        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

def convert_to_bst(dt):
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')  # BST timezone for London

    utc_dt = utc_tz.localize(dt)
    bst_dt = utc_dt.astimezone(bst_tz)

    return bst_dt.strftime('%H:%M:%S')

def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}

        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors

        data = response.json()

        if len(data) == 0:
            print("No arrival predictions found.")
            return None, None

        station_name = data[0]['stationName']

        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)  # Convert GMT to BST
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')  # Format to HH:MM
            })

        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))

        return df, station_name

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None  # Return None if there's an error

def create_dashboard(df, station_name):
    current_time_utc = datetime.utcnow()
    current_time_bst = convert_to_bst(current_time_utc)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=[current_time_bst],
        y=[0],
        mode='markers+text',
        marker=dict(size=40, symbol='square', color='black'),
        name='Stop Point',
        text=f"{current_time_bst}",  # Display only the current time below the square
        textposition='bottom center',
        hoverinfo='text'
    ))

    fig.add_annotation(
        x=current_time_bst,
        y=0,
        text=f"{station_name}<br>{stop_point_id}",
        showarrow=False,
        font=dict(size=14, color="black"),
        yshift=40  # Adjust the yshift to move the label above the square
    )

    y_gap = 0.01  # 1 cm gap in graph units
    for i, row in df.iterrows():
        y_pos = 0 + (i + 1) * y_gap

        fig.add_trace(go.Scatter(
            x=[row['Expected Arrival (BST)']],
            y=[y_pos],
            mode='markers+text',
            marker=dict(size=20, symbol='arrow-bar-down', color='blue'),
            name='Bus Arrival',
            text=f"Bus ID: {row['Vehicle ID']}<br>Arrival Time: {row['Expected Arrival (BST)']}",
            textposition='top center',
            hoverinfo='text'
        ))

    fig.update_layout(
        title=f"Expected Bus Arrivals at {station_name}",
        xaxis_title="Time",
        yaxis_title="",
        yaxis=dict(showticklabels=False),
        showlegend=False,
        height=400,
        margin=dict(l=0, r=0, t=40, b=0)
    )

    fig.show()

def calculate_summary_metrics(dfs):
    concatenated_df = pd.concat(dfs)
    summary = concatenated_df.describe(include='all')
    return summary

# Sample Inputs
line_id = input("Enter the Line ID (e.g., 205, 25): ")
direction = input("Enter the direction (inbound or outbound): ")
buses_per_hour = int(input("Enter the number of buses per hour: "))

fetch_route_details(line_id, direction)

df_stops = pd.read_csv('stop_details.csv')

print(df_stops)

selected_stop_input = input("Enter the row numbers, stop IDs, or stop names of the stops you're interested in (comma separated): ")
selected_stops = [item.strip() for item in selected_stop_input.split(',')]

matching_stops = pd.DataFrame()

for stop in selected_stops:
    if stop.isdigit():
        matching_stops = matching_stops.append(df_stops.loc[df_stops['Sl. No.'] == int(stop)], ignore_index=True)
    else:
        matching_stops = matching_stops.append(
            df_stops.loc[
                (df_stops['Stop ID'].str.contains(stop, case=False)) |
                (df_stops['Stop Name'].str.contains(stop, case=False))
            ], ignore_index=True
        )

if matching_stops.empty:
    print("No matching stops found.")
else:
    df_predictions_list = []
    for index, stop_row in matching_stops.iterrows():
        stop_point_id = stop_row['Stop ID']
        stop_point_name = stop_row['Stop Name']
        df_predictions, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)
        if df_predictions is not None:
            df_predictions_list.append(df_predictions)
            create_dashboard(df_predictions, stop_point_name)

    if df_predictions_list:
        summary_metrics = calculate_summary_metrics(df_predictions_list)
        print("Summary Metrics:")
        print(summary_metrics)


Enter the Line ID (e.g., 205, 25): 214
Enter the direction (inbound or outbound): inbound
Enter the number of buses per hour: 6
Direction: inbound
Origination Name: Finsbury Square
Destination Name: Highgate School
Originator: 490015209C
Destination: 490008160N
---
Data written to 'stop_details.csv' successfully.
Data from CSV:
1      490015209K Finsbury Square / Moorgate
2      490015209C Finsbury Square
3      490006850J Finsbury Square / Moorgate
4      490006850G Finsbury Square
5      490006850H Finsbury Square
6      490003831E Finsbury Street
7      490015209D Finsbury Square
8      490003831W Finsbury Street
9      490000149B Moorgate Station
10     490000149M Moorgate
11     490006850F Finsbury Square
12     490006612E Epworth Street
13     490000169ZA Old Street Station
14     490015195N Old Street
15     490015195M City Road / Leonard Street
16     490000169K Shoreditch Fire Station
17     490015194F Old Street Station
18     490000169L Old Street Station
19     490010109E O

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
import requests
import csv
import re
import time
from datetime import datetime, timedelta
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pytz

def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {'serviceTypes': 'Regular'}

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        route_sections = data.get('routeSections', [])

        found_direction = False

        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')

                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")

                # Fetch and print sequence of stop points
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)

                if all_stop_details:
                    # Write to CSV file
                    write_to_csv(all_stop_details)

                    # Display data from CSV file
                    display_data_from_csv()

        if not found_direction:
            print(f"No route found for direction '{direction}'.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        line_strings = data.get('lineStrings', [])

        if line_strings:
            printed_stop_ids = set()  # To track printed stop IDs

            for line_string in line_strings:
                # Parsing the line string to extract latitudes and longitudes
                coordinates = eval(line_string)  # Evaluating string to list
                for lon, lat in coordinates[0]:
                    # Fetching and appending stop details using the lat and lon
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)

        else:
            print("No stop points found for this route.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")

    return all_stop_details

def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"

    try:
        for attempt in range(retries):
            response = requests.get(url)

            if response.status_code == 429:
                time.sleep(2**attempt)  # Exponential backoff
                continue

            response.raise_for_status()  # Raise exception for bad status codes

            data = response.json()
            stop_points = data.get('stopPoints', [])

            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')

                    # Append to all_stop_details only if stop ID has not been printed before
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')

                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })

                        # Add stop ID to printed set
                        printed_stop_ids.add(stop_id)

            else:
                # If no stop points found, continue to next attempt
                continue

            break  # Exit loop if successful

        else:
            # If maximum retries exceeded, handle quietly (you can add logging or alternative handling here)
            pass

    except requests.exceptions.RequestException as e:
        # Handle other request exceptions (you can add logging or alternative handling here)
        pass

def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']

    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()

        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)

    print(f"Data written to 'stop_details.csv' successfully.")

def display_data_from_csv():
    print("Data from CSV:")

    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)

        # Print headers
        headers_printed = False

        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

def convert_to_bst(dt):
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')  # BST timezone for London

    utc_dt = utc_tz.localize(dt)
    bst_dt = utc_dt.astimezone(bst_tz)

    return bst_dt.strftime('%H:%M:%S')

def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}

        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors

        data = response.json()

        if len(data) == 0:
            print("No arrival predictions found.")
            return None, None

        station_name = data[0]['stationName']

        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)  # Convert GMT to BST
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')  # Format to HH:MM
            })

        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))

        return df, station_name

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None  # Return None if there's an error

def create_dashboard(df, station_name):
    current_time_utc = datetime.utcnow()
    current_time_bst = convert_to_bst(current_time_utc)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=[current_time_bst],
        y=[0],
        mode='markers+text',
        marker=dict(size=40, symbol='square', color='black'),
        name='Stop Point',
        text=f"{current_time_bst}",  # Display only the current time below the square
        textposition='bottom center',
        hoverinfo='text'
    ))

    fig.add_annotation(
        x=current_time_bst,
        y=0,
        text=f"{station_name}<br>{stop_point_id}",
        showarrow=False,
        font=dict(size=14, color="black"),
        yshift=40  # Adjust the yshift to move the label above the square
    )

    y_gap = 0.01  # 1 cm gap in graph units
    for i, row in df.iterrows():
        y_pos = 0 + (i + 1) * y_gap

        fig.add_trace(go.Scatter(
            x=[row['Expected Arrival (BST)']],
            y=[y_pos],
            mode='markers+text',
            marker=dict(size=20, symbol='arrow-bar-down', color='blue'),
            name='Bus Arrival',
            text=f"Bus ID: {row['Vehicle ID']}<br>Arrival Time: {row['Expected Arrival (BST)']}",
            textposition='top center',
            hoverinfo='text'
        ))

    fig.update_layout(
        title=f"Expected Bus Arrivals at {station_name}",
        xaxis_title="Time",
        yaxis_title="",
        yaxis=dict(showticklabels=False),
        showlegend=False,
        height=400,
        margin=dict(l=0, r=0, t=40, b=0)
    )

    fig.show()

def calculate_summary_metrics(dfs):
    concatenated_df = pd.concat(dfs)
    summary = concatenated_df.describe(include='all')
    return summary

# Sample Inputs
line_id = input("Enter the Line ID (e.g., 205, 25): ")
direction = input("Enter the direction (inbound or outbound): ")
buses_per_hour = int(input("Enter the number of buses per hour: "))

fetch_route_details(line_id, direction)

df_stops = pd.read_csv('stop_details.csv')

print(df_stops)

selected_stop_input = input("Enter the row numbers, stop IDs, or stop names of the stops you're interested in (comma separated): ")
selected_stops = [item.strip() for item in selected_stop_input.split(',')]

matching_stops = pd.DataFrame()

for stop in selected_stops:
    if stop.isdigit():
        matching_stops = pd.concat([matching_stops, df_stops.loc[df_stops['Sl. No.'] == int(stop)]], ignore_index=True)
    else:
        matching_stops = pd.concat([
            matching_stops,
            df_stops.loc[
                (df_stops['Stop ID'].str.contains(stop, case=False)) |
                (df_stops['Stop Name'].str.contains(stop, case=False))
            ]
        ], ignore_index=True)

if matching_stops.empty:
    print("No matching stops found.")
else:
    df_predictions_list = []
    for index, stop_row in matching_stops.iterrows():
        stop_point_id = stop_row['Stop ID']
        stop_point_name = stop_row['Stop Name']
        df_predictions, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)
        if df_predictions is not None:
            df_predictions_list.append(df_predictions)
            create_dashboard(df_predictions, stop_point_name)

    if df_predictions_list:
        summary_metrics = calculate_summary_metrics(df_predictions_list)
        print("Summary Metrics:")
        print(summary_metrics)


Enter the Line ID (e.g., 205, 25): 214
Enter the direction (inbound or outbound): inbound
Enter the number of buses per hour: 6
Direction: inbound
Origination Name: Finsbury Square
Destination Name: Highgate School
Originator: 490015209C
Destination: 490008160N
---
Data written to 'stop_details.csv' successfully.
Data from CSV:
1      490015209K Finsbury Square / Moorgate
2      490015209C Finsbury Square
3      490006850J Finsbury Square / Moorgate
4      490006850G Finsbury Square
5      490006850H Finsbury Square
6      490003831E Finsbury Street
7      490015209D Finsbury Square
8      490003831W Finsbury Street
9      490000149B Moorgate Station
10     490000149M Moorgate
11     490006850F Finsbury Square
12     490006612E Epworth Street
13     490000169ZA Old Street Station
14     490015195N Old Street
15     490015195M City Road / Leonard Street
16     490000169K Shoreditch Fire Station
17     490015194F Old Street Station
18     490000169L Old Street Station
19     490010109E O

Summary Metrics:
       Line Vehicle ID  Stop Point Direction Expected Arrival (BST)  \
count     5          5           5         5                      5   
unique    1          5           1         1                      5   
top     214    LA19KBN  490010012N   inbound               17:28:00   
freq      5          1           5         5                      1   
mean    NaN        NaN         NaN       NaN                    NaN   
min     NaN        NaN         NaN       NaN                    NaN   
25%     NaN        NaN         NaN       NaN                    NaN   
50%     NaN        NaN         NaN       NaN                    NaN   
75%     NaN        NaN         NaN       NaN                    NaN   
max     NaN        NaN         NaN       NaN                    NaN   
std     NaN        NaN         NaN       NaN                    NaN   

       Expected Arrival (HM)  Headway (minutes)  AWT/bus (minutes)       WAWT  
count                      5           5.000000   

In [ ]:
import requests
import csv
import re
import time
from datetime import datetime, timedelta
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pytz

def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {'serviceTypes': 'Regular'}

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        route_sections = data.get('routeSections', [])

        found_direction = False

        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')

                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")

                # Fetch and print sequence of stop points
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)

                if all_stop_details:
                    # Write to CSV file
                    write_to_csv(all_stop_details)

                    # Display data from CSV file
                    display_data_from_csv()

        if not found_direction:
            print(f"No route found for direction '{direction}'.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        line_strings = data.get('lineStrings', [])

        if line_strings:
            printed_stop_ids = set()  # To track printed stop IDs

            for line_string in line_strings:
                # Parsing the line string to extract latitudes and longitudes
                coordinates = eval(line_string)  # Evaluating string to list
                for lon, lat in coordinates[0]:
                    # Fetching and appending stop details using the lat and lon
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)

        else:
            print("No stop points found for this route.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")

    return all_stop_details

def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"

    try:
        for attempt in range(retries):
            response = requests.get(url)

            if response.status_code == 429:
                time.sleep(2**attempt)  # Exponential backoff
                continue

            response.raise_for_status()  # Raise exception for bad status codes

            data = response.json()
            stop_points = data.get('stopPoints', [])

            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')

                    # Append to all_stop_details only if stop ID has not been printed before
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')

                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })

                        # Add stop ID to printed set
                        printed_stop_ids.add(stop_id)

            else:
                # If no stop points found, continue to next attempt
                continue

            break  # Exit loop if successful

        else:
            # If maximum retries exceeded, handle quietly (you can add logging or alternative handling here)
            pass

    except requests.exceptions.RequestException as e:
        # Handle other request exceptions (you can add logging or alternative handling here)
        pass

def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']

    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()

        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)

    print(f"Data written to 'stop_details.csv' successfully.")

def display_data_from_csv():
    print("Data from CSV:")

    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)

        # Print headers
        headers_printed = False

        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

def convert_to_bst(dt):
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')  # BST timezone for London

    utc_dt = utc_tz.localize(dt)
    bst_dt = utc_dt.astimezone(bst_tz)

    return bst_dt.strftime('%H:%M:%S')

def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}

        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors

        data = response.json()

        if len(data) == 0:
            print("No arrival predictions found.")
            return None, None

        station_name = data[0]['stationName']

        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)  # Convert GMT to BST
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')  # Format to HH:MM
            })

        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))

        return df, station_name

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None  # Return None if there's an error

def create_dashboard(df, station_name):
    current_time_utc = datetime.utcnow()
    current_time_bst = convert_to_bst(current_time_utc)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=[current_time_bst],
        y=[0],
        mode='markers+text',
        marker=dict(size=40, symbol='square', color='black'),
        name='Stop Point',
        text=f"{current_time_bst}",  # Display only the current time below the square
        textposition='bottom center',
        hoverinfo='text'
    ))

    fig.add_annotation(
        x=current_time_bst,
        y=0,
        text=f"{station_name}",
        showarrow=False,
        font=dict(size=14, color="black"),
        yshift=40  # Adjust the yshift to move the label above the square
    )

    y_gap = 0.01  # 1 cm gap in graph units
    for i, row in df.iterrows():
        y_pos = 0 + (i + 1) * y_gap

        fig.add_trace(go.Scatter(
            x=[row['Expected Arrival (BST)']],
            y=[y_pos],
            mode='markers+text',
            marker=dict(size=20, symbol='arrow-bar-down', color='blue'),
            name='Bus Arrival',
            text=f"Bus ID: {row['Vehicle ID']}<br>Arrival Time: {row['Expected Arrival (BST)']}",
            textposition='top center',
            hoverinfo='text'
        ))

    fig.update_layout(
        title=f"Expected Bus Arrivals at {station_name}",
        xaxis_title="Time",
        yaxis_title="",
        yaxis=dict(showticklabels=False),
        showlegend=False,
        height=400,
        margin=dict(l=0, r=0, t=40, b=0)
    )

    fig.show()

def calculate_summary_metrics(df):
    summary = df.describe(include='all')
    return summary

# Sample Inputs
line_id = input("Enter the Line ID (e.g., 205, 25): ")
direction = input("Enter the direction (inbound or outbound): ")
buses_per_hour = int(input("Enter the number of buses per hour: "))

fetch_route_details(line_id, direction)

df_stops = pd.read_csv('stop_details.csv')

print(df_stops)

num_stops = int(input("Enter the number of stops of interest: "))

selected_stops = []
for i in range(num_stops):
    selected_stop = input(f"Enter row number, stop ID, or stop name for stop {i + 1}: ").strip()
    selected_stops.append(selected_stop)

matching_stops = pd.DataFrame()

for stop in selected_stops:
    if stop.isdigit():
        matching_stops = pd.concat([matching_stops, df_stops.loc[df_stops['Sl. No.'] == int(stop)]], ignore_index=True)
    else:
        matching_stops = pd.concat([
            matching_stops,
            df_stops.loc[
                (df_stops['Stop ID'].str.contains(stop, case=False)) |
                (df_stops['Stop Name'].str.contains(stop, case=False))
            ]
        ], ignore_index=True)

if matching_stops.empty:
    print("No matching stops found.")
else:
    for index, stop_row in matching_stops.iterrows():
        stop_point_id = stop_row['Stop ID']
        stop_point_name = stop_row['Stop Name']
        df_predictions, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)
        if df_predictions is not None:
            create_dashboard(df_predictions, station_name)

            summary_metrics = calculate_summary_metrics(df_predictions)
            print(f"\nSummary Metrics for {station_name}:")
            print(summary_metrics)


Direction: inbound
Origination Name: Finsbury Square
Destination Name: Highgate School
Originator: 490015209C
Destination: 490008160N
---
Data written to 'stop_details.csv' successfully.
Data from CSV:
1      490015209K Finsbury Square / Moorgate
2      490015209C Finsbury Square
3      490006850J Finsbury Square / Moorgate
4      490006850G Finsbury Square
5      490006850H Finsbury Square
6      490003831E Finsbury Street
7      490015209D Finsbury Square
8      490003831W Finsbury Street
9      490000149B Moorgate Station
10     490000149M Moorgate
11     490006850F Finsbury Square
12     490006612E Epworth Street
13     490000169ZA Old Street Station
14     490015195N Old Street
15     490015195M City Road / Leonard Street
16     490000169K Shoreditch Fire Station
17     490015194F Old Street Station
18     490000169L Old Street Station
19     490010109E Old Street Roundabout
20     490015194G Bunhill Row
21     490015193S Old Street Station
22     490015193D Old Street Stn   / Moo

KeyboardInterrupt: Interrupted by user

In [ ]:
import requests
import csv
import re
import time
from datetime import datetime, timedelta
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pytz

def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {'serviceTypes': 'Regular'}

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        route_sections = data.get('routeSections', [])

        found_direction = False

        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')

                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")

                # Fetch and print sequence of stop points
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)

                if all_stop_details:
                    # Write to CSV file
                    write_to_csv(all_stop_details)

                    # Display data from CSV file
                    display_data_from_csv()

        if not found_direction:
            print(f"No route found for direction '{direction}'.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        line_strings = data.get('lineStrings', [])

        if line_strings:
            printed_stop_ids = set()  # To track printed stop IDs

            for line_string in line_strings:
                # Parsing the line string to extract latitudes and longitudes
                coordinates = eval(line_string)  # Evaluating string to list
                for lon, lat in coordinates:
                    # Fetching and appending stop details using the lat and lon
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)

        else:
            print("No stop points found for this route.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")

    return all_stop_details

def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/StopPoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"

    try:
        for attempt in range(retries):
            response = requests.get(url)

            if response.status_code == 429:
                time.sleep(2**attempt)  # Exponential backoff
                continue

            response.raise_for_status()  # Raise exception for bad status codes

            data = response.json()
            stop_points = data.get('stopPoints', [])

            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')

                    # Append to all_stop_details only if stop ID has not been printed before
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')

                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })

                        # Add stop ID to printed set
                        printed_stop_ids.add(stop_id)

            else:
                # If no stop points found, continue to next attempt
                continue

            break  # Exit loop if successful

        else:
            # If maximum retries exceeded, handle quietly (you can add logging or alternative handling here)
            pass

    except requests.exceptions.RequestException as e:
        # Handle other request exceptions (you can add logging or alternative handling here)
        pass

def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']

    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()

        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)

    print(f"Data written to 'stop_details.csv' successfully.")

def display_data_from_csv():
    print("Data from CSV:")

    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)

        # Print headers
        headers_printed = False

        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

def convert_to_bst(dt):
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')  # BST timezone for London

    utc_dt = utc_tz.localize(dt)
    bst_dt = utc_dt.astimezone(bst_tz)

    return bst_dt.strftime('%H:%M:%S')

def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}

        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors

        data = response.json()

        if len(data) == 0:
            print("No arrival predictions found.")
            return None, None

        station_name = data[0]['stationName']

        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)  # Convert GMT to BST
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')  # Format to HH:MM
            })

        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))

        return df, station_name

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None  # Return None if there's an error

def create_dashboard(df, station_name):
    current_time_utc = datetime.utcnow()
    current_time_bst = convert_to_bst(current_time_utc)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=[current_time_bst],
        y=[0],
        mode='markers+text',
        marker=dict(size=40, symbol='square', color='black'),
        name='Stop Point',
        text=f"{current_time_bst}",  # Display only the current time below the square
        textposition='bottom center',
        hoverinfo='text'
    ))

    fig.add_annotation(
        x=current_time_bst,
        y=0,
        text=f"{station_name}",
        showarrow=False,
        font=dict(size=14, color="black"),
        yshift=40  # Adjust the yshift to move the label above the square
    )

    y_gap = 0.01  # 1 cm gap in graph units
    for i, row in df.iterrows():
        y_pos = 0 + (i + 1) * y_gap

        fig.add_trace(go.Scatter(
            x=[row['Expected Arrival (BST)']],
            y=[y_pos],
            mode='markers+text',
            marker=dict(size=20, symbol='arrow-bar-down', color='blue'),
            name='Bus Arrival',
            text=f"Bus ID: {row['Vehicle ID']}<br>Arrival Time: {row['Expected Arrival (BST)']}",
            textposition='top center',
            hoverinfo='text'
        ))

    fig.update_layout(
        title=f"Expected Bus Arrivals at {station_name}",
        xaxis_title="Time",
        yaxis_title="",
        yaxis=dict(showticklabels=False),
        showlegend=False,
        height=400,
        margin=dict(l=0, r=0, t=40, b=0)
    )

    fig.show()

def calculate_summary_metrics(df):
    summary = df.describe(include='all')
    return summary

# Sample Inputs
line_id = input("Enter the Line ID (e.g., 205, 25): ")
direction = input("Enter the direction (inbound or outbound): ")
buses_per_hour = int(input("Enter the number of buses per hour: "))

fetch_route_details(line_id, direction)

df_stops = pd.read_csv('stop_details.csv')

print(df_stops)

num_stops = int(input("Enter the number of stops of interest: "))

selected_stops = []
for i in range(num_stops):
    selected_stop = input(f"Enter row number, stop ID, or stop name for stop {i + 1}: ").strip()
    selected_stops.append(selected_stop)

matching_stops = pd.DataFrame()

for stop in selected_stops:
    if stop.isdigit():
        matching_stops = pd.concat([matching_stops, df_stops.loc[df_stops['Sl. No.'] == int(stop)]], ignore_index=True)
    else:
        matching_stops = pd.concat([
            matching_stops,
            df_stops.loc[
                (df_stops['Stop ID'].str.contains(stop, case=False)) |
                (df_stops['Stop Name'].str.contains(stop, case=False))
            ]
        ], ignore_index=True)

if matching_stops.empty:
    print("No matching stops found.")
else:
    for index, stop_row in matching_stops.iterrows():
        stop_point_id = stop_row['Stop ID']
        stop_point_name = stop_row['Stop Name']
        df_predictions, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)
        if df_predictions is not None:
            create_dashboard(df_predictions, station_name)

            summary_metrics = calculate_summary_metrics(df_predictions)
            print(f"\nSummary Metrics for {station_name}:")
            print(summary_metrics)


Enter the Line ID (e.g., 205, 25): 214
Enter the direction (inbound or outbound): inbound
Enter the number of buses per hour: 6
Direction: inbound
Origination Name: Finsbury Square
Destination Name: Highgate School
Originator: 490015209C
Destination: 490008160N
---


ValueError: too many values to unpack (expected 2)

In [ ]:
import requests
import csv
import re
import time
from datetime import datetime, timedelta
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pytz

def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {'serviceTypes': 'Regular'}

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        route_sections = data.get('routeSections', [])

        found_direction = False

        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')

                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")

                # Fetch and print sequence of stop points
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)

                if all_stop_details:
                    # Write to CSV file
                    write_to_csv(all_stop_details)

                    # Display data from CSV file
                    display_data_from_csv()

        if not found_direction:
            print(f"No route found for direction '{direction}'.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        line_strings = data.get('lineStrings', [])

        if line_strings:
            printed_stop_ids = set()  # To track printed stop IDs

            for line_string in line_strings:
                # Parsing the line string to extract latitudes and longitudes
                coordinates = eval(line_string)  # Evaluating string to list
                for coord_pair in coordinates:
                    lon, lat = coord_pair  # Unpack lon, lat from coord_pair
                    # Fetching and appending stop details using the lat and lon
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)

        else:
            print("No stop points found for this route.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")

    return all_stop_details

def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/StopPoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"

    try:
        for attempt in range(retries):
            response = requests.get(url)

            if response.status_code == 429:
                time.sleep(2**attempt)  # Exponential backoff
                continue

            response.raise_for_status()  # Raise exception for bad status codes

            data = response.json()
            stop_points = data.get('stopPoints', [])

            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')

                    # Append to all_stop_details only if stop ID has not been printed before
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')

                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })

                        # Add stop ID to printed set
                        printed_stop_ids.add(stop_id)

            else:
                # If no stop points found, continue to next attempt
                continue

            break  # Exit loop if successful

        else:
            # If maximum retries exceeded, handle quietly (you can add logging or alternative handling here)
            pass

    except requests.exceptions.RequestException as e:
        # Handle other request exceptions (you can add logging or alternative handling here)
        pass

def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']

    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()

        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)

    print(f"Data written to 'stop_details.csv' successfully.")

def display_data_from_csv():
    print("Data from CSV:")

    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)

        # Print headers
        headers_printed = False

        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

def convert_to_bst(dt):
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')  # BST timezone for London

    utc_dt = utc_tz.localize(dt)
    bst_dt = utc_dt.astimezone(bst_tz)

    return bst_dt.strftime('%H:%M:%S')

def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}

        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors

        data = response.json()

        if len(data) == 0:
            print("No arrival predictions found.")
            return None, None

        station_name = data[0]['stationName']

        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)  # Convert GMT to BST
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')  # Format to HH:MM
            })

        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))

        return df, station_name

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None  # Return None if there's an error

def create_dashboard(df, station_name):
    current_time_utc = datetime.utcnow()
    current_time_bst = convert_to_bst(current_time_utc)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=[current_time_bst],
        y=[0],
        mode='markers+text',
        marker=dict(size=40, symbol='square', color='black'),
        name='Stop Point',
        text=f"{current_time_bst}",  # Display only the current time below the square
        textposition='bottom center',
        hoverinfo='text'
    ))

    fig.add_annotation(
        x=current_time_bst,
        y=0,
        text=f"{station_name}",
        showarrow=False,
        font=dict(size=14, color="black"),
        yshift=40  # Adjust the yshift to move the label above the square
    )

    y_gap = 0.01  # 1 cm gap in graph units
    for i, row in df.iterrows():
        y_pos = 0 + (i + 1) * y_gap

        fig.add_trace(go.Scatter(
            x=[row['Expected Arrival (BST)']],
            y=[y_pos],
            mode='markers+text',
            marker=dict(size=20, symbol='arrow-bar-down', color='blue'),
            name='Bus Arrival',
            text=f"Bus ID: {row['Vehicle ID']}<br>Arrival Time: {row['Expected Arrival (BST)']}",
            textposition='top center',
            hoverinfo='text'
        ))

    fig.update_layout(
        title=f"Expected Bus Arrivals at {station_name}",
        xaxis_title="Time",
        yaxis_title="",
        yaxis=dict(showticklabels=False),
        showlegend=False,
        height=400,
        margin=dict(l=0, r=0, t=40, b=0)
    )

    fig.show()

def calculate_summary_metrics(df):
    summary = df.describe(include='all')
    return summary

# Sample Inputs
line_id = input("Enter the Line ID (e.g., 205, 25): ")
direction = input("Enter the direction (inbound or outbound): ")
buses_per_hour = int(input("Enter the number of buses per hour: "))

fetch_route_details(line_id, direction)

df_stops = pd.read_csv('stop_details.csv')

print(df_stops)

num_stops = int(input("Enter the number of stops of interest: "))

selected_stops = []
for i in range(num_stops):
    selected_stop = input(f"Enter row number, stop ID, or stop name for stop {i + 1}: ").strip()
    selected_stops.append(selected_stop)

matching_stops = pd.DataFrame()

for stop in selected_stops:
    if stop.isdigit():
        matching_stops = pd.concat([matching_stops, df_stops.loc[df_stops['Sl. No.'] == int(stop)]], ignore_index=True)
    else:
        matching_stops = pd.concat([
            matching_stops,
            df_stops.loc[
                (df_stops['Stop ID'].str.contains(stop, case=False)) |
                (df_stops['Stop Name'].str.contains(stop, case=False))
            ]
        ], ignore_index=True)

if matching_stops.empty:
    print("No matching stops found.")
else:
    for index, stop_row in matching_stops.iterrows():
        stop_point_id = stop_row['Stop ID']
        stop_point_name = stop_row['Stop Name']
        df_predictions, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)
        if df_predictions is not None:
            create_dashboard(df_predictions, station_name)

            summary_metrics = calculate_summary_metrics(df_predictions)
            print(f"\nSummary Metrics for {station_name}:")
            print(summary_metrics)


Enter the Line ID (e.g., 205, 25): 214
Enter the direction (inbound or outbound): inbound
Enter the number of buses per hour: 6
Direction: inbound
Origination Name: Finsbury Square
Destination Name: Highgate School
Originator: 490015209C
Destination: 490008160N
---


ValueError: too many values to unpack (expected 2)

25/06/2024

In [1]:
import requests
import csv
import re
import time

def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {
        'serviceTypes': 'Regular'
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        route_sections = data.get('routeSections', [])

        found_direction = False

        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')

                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")

                # Fetch and print sequence of stop points
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)

                if all_stop_details:
                    # Write to CSV file
                    write_to_csv(all_stop_details)

                    # Display data from CSV file
                    display_data_from_csv()

                    # Get QSI stop points from the user
                    get_qsi_stop_points()

        if not found_direction:
            print(f"No route found for direction '{direction}'.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        line_strings = data.get('lineStrings', [])

        if line_strings:
            printed_stop_ids = set()  # To track printed stop IDs

            for line_string in line_strings:
                # Parsing the line string to extract latitudes and longitudes
                coordinates = eval(line_string)  # Evaluating string to list
                for lon, lat in coordinates[0]:
                    # Fetching and appending stop details using the lat and lon
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)

        else:
            print("No stop points found for this route.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")

    return all_stop_details

def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"

    try:
        for attempt in range(retries):
            response = requests.get(url)

            if response.status_code == 429:
                time.sleep(2**attempt)  # Exponential backoff
                continue

            response.raise_for_status()  # Raise exception for bad status codes

            data = response.json()
            stop_points = data.get('stopPoints', [])

            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')

                    # Append to all_stop_details only if stop ID has not been printed before
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')

                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })

                        # Add stop ID to printed set
                        printed_stop_ids.add(stop_id)

            else:
                # If no stop points found, continue to next attempt
                continue

            break  # Exit loop if successful

        else:
            # If maximum retries exceeded, handle quietly (you can add logging or alternative handling here)
            pass

    except requests.exceptions.RequestException as e:
        # Handle other request exceptions (you can add logging or alternative handling here)
        pass

def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']

    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()

        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)

    print(f"Data written to 'stop_details.csv' successfully.")

def display_data_from_csv():
    print("Data from CSV:")

    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)

        # Print headers
        headers_printed = False

        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

def get_qsi_stop_points():
    count_of_qsi_stop_points = int(input("Enter the number of QSI stop points: "))
    qsi_stop_points = []

    for i in range(count_of_qsi_stop_points):
        row_no = int(input(f"Enter the row number for QSI stop point {i+1}: "))
        qsi_stop_points.append(row_no)

    # Display the QSI stop points
    print("QSI Stop Points:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if int(row['Sl. No.']) in qsi_stop_points:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

# Example usage
if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")

    fetch_route_details(line_id, direction)


Enter line ID: 214
Enter direction (e.g., inbound or outbound): inbound
Direction: inbound
Origination Name: Finsbury Square
Destination Name: Highgate School
Originator: 490015209C
Destination: 490008160N
---
Data written to 'stop_details.csv' successfully.
Data from CSV:
1      490015209K Finsbury Square / Moorgate
2      490015209C Finsbury Square
3      490006850J Finsbury Square / Moorgate
4      490006850G Finsbury Square
5      490006850H Finsbury Square
6      490003831E Finsbury Street
7      490015209D Finsbury Square
8      490003831W Finsbury Street
9      490000149B Moorgate Station
10     490000149M Moorgate
11     490006850F Finsbury Square
12     490006612E Epworth Street
13     490000169ZA Old Street Station
14     490015195N Old Street
15     490015195M City Road / Leonard Street
16     490000169K Shoreditch Fire Station
17     490015194F Old Street Station
18     490000169L Old Street Station
19     490010109E Old Street Roundabout
20     490015194G Bunhill Row
21   

Combined code 1

In [3]:
# Step 2: Install required libraries (if not already installed)
!pip install requests pandas plotly dash

# Step 3: Import required libraries
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from IPython.display import display, clear_output
import plotly.express as px
import plotly.graph_objects as go
import pytz
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Function to convert time to BST
def convert_to_bst(dt):
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')
    utc_dt = utc_tz.localize(dt)
    bst_dt = utc_dt.astimezone(bst_tz)
    return bst_dt.strftime('%H:%M:%S')

# Function to fetch arrival predictions with error handling
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + pd.Timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

# Fetch route details
def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {'serviceTypes': 'Regular'}
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        route_sections = data.get('routeSections', [])
        found_direction = False
        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')
                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)
                if all_stop_details:
                    write_to_csv(all_stop_details)
                    display_data_from_csv()
                    get_qsi_stop_points()
        if not found_direction:
            print(f"No route found for direction '{direction}'.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

# Fetch and append stop points
def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        line_strings = data.get('lineStrings', [])
        if line_strings:
            printed_stop_ids = set()
            for line_string in line_strings:
                coordinates = eval(line_string)
                for lon, lat in coordinates[0]:
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)
        else:
            print("No stop points found for this route.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")
    return all_stop_details

# Fetch and append stop details
def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"
    try:
        for attempt in range(retries):
            response = requests.get(url)
            if response.status_code == 429:
                time.sleep(2**attempt)
                continue
            response.raise_for_status()
            data = response.json()
            stop_points = data.get('stopPoints', [])
            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')
                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })
                        printed_stop_ids.add(stop_id)
            else:
                continue
            break
        else:
            pass
    except requests.exceptions.RequestException as e:
        pass

# Write to CSV
def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']
    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()
        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)
    print(f"Data written to 'stop_details.csv' successfully.")

# Display data from CSV
def display_data_from_csv():
    print("Data from CSV:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        headers_printed = False
        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

# Get QSI stop points
def get_qsi_stop_points():
    count_of_qsi_stop_points = int(input("Enter the number of QSI stop points: "))
    qsi_stop_points = []
    for i in range(count_of_qsi_stop_points):
        row_no = int(input(f"Enter the row number for QSI stop point {i+1}: "))
        qsi_stop_points.append(row_no)
    qsi_stop_ids = []
    print("QSI Stop Points:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if int(row['Sl. No.']) in qsi_stop_points:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                qsi_stop_ids.append(row['Stop ID'])
    return qsi_stop_ids

# Main loop to fetch and display data
def main_loop(line_id, direction, qsi_stop_ids):
    while True:
        for idx, stop_point_id in enumerate(qsi_stop_ids):
            df, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)
            if df is not None and station_name is not None:
                clear_output(wait=True)
                total_wawt = df['WAWT'].sum()
                min_arrival = pd.to_datetime(df['Expected Arrival (BST)']).min().replace(second=0, microsecond=0)
                max_arrival = pd.to_datetime(df['Expected Arrival (BST)']).max().replace(second=0, microsecond=0)
                time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60
                awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0
                nbph = len(df) / (time_diff_minutes / 60)
                swt = round(60 / (nbph * 2), 2)
                ewt = round(awt - swt, 2)
                summary_df = pd.DataFrame({
                    'Metric': ['Number of buses scheduled per hour (nbph)', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses (minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
                    'Value': [nbph, total_wawt, time_diff_minutes, awt, swt, ewt]
                })
                print(f"Arrival Predictions for QSI Stop Point {idx + 1} ({station_name}):")
                display(df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']])
                print("\nSummary Metrics:")
                display(summary_df)
            else:
                print(f"Failed to fetch data for QSI Stop Point {idx + 1}.")
        time.sleep(15)

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    fetch_route_details(line_id, direction)
    qsi_stop_ids = get_qsi_stop_points()
    main_loop(line_id, direction, qsi_stop_ids)


Arrival Predictions for QSI Stop Point 1 (Moorfields Eye Hospital):


<ipython-input-3-46a2688d1635>:191: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  min_arrival = pd.to_datetime(df['Expected Arrival (BST)']).min().replace(second=0, microsecond=0)
<ipython-input-3-46a2688d1635>:192: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  max_arrival = pd.to_datetime(df['Expected Arrival (BST)']).max().replace(second=0, microsecond=0)


,Line,Vehicle ID,Stop Point,Direction,Expected Arrival (BST),Headway (minutes),AWT/bus (minutes),WAWT
3,214,LA19KCE,490010012N,inbound,09:43:00,0.0,0.0,0.0
2,214,LA19KBZ,490010012N,inbound,09:50:00,7.0,3.5,24.5
0,214,LA19KBU,490010012N,inbound,09:58:00,8.0,4.0,32.0
1,214,LA19KBV,490010012N,inbound,10:07:00,9.0,4.5,40.5



Summary Metrics:


,Metric,Value
0,Number of buses scheduled per hour (nbph),10.00
1,Total WAWT (minutes),97.00
2,Time difference between 1st and last observed ...,24.00
3,AWT (minutes),4.04
4,SWT (minutes),3.00
5,EWT (minutes),1.04


IndexError: list index out of range

In [2]:
# Step 2: Install required libraries (if not already installed)
!pip install requests pandas plotly dash

# Step 3: Import required libraries
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from IPython.display import display, clear_output
import plotly.express as px
import plotly.graph_objects as go
import pytz
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import csv

# Function to convert time to BST
def convert_to_bst(dt):
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')
    utc_dt = utc_tz.localize(dt)
    bst_dt = utc_dt.astimezone(bst_tz)
    return bst_dt.strftime('%H:%M:%S')

# Function to fetch arrival predictions with error handling
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None  # No data available
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

# Fetch route details
def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {'serviceTypes': 'Regular'}
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        route_sections = data.get('routeSections', [])
        found_direction = False
        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')
                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)
                if all_stop_details:
                    write_to_csv(all_stop_details)
                    display_data_from_csv()
        if not found_direction:
            print(f"No route found for direction '{direction}'.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

# Fetch and append stop points
def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        line_strings = data.get('lineStrings', [])
        if line_strings:
            printed_stop_ids = set()
            for line_string in line_strings:
                coordinates = eval(line_string)
                for lon, lat in coordinates[0]:
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)
        else:
            print("No stop points found for this route.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")
    return all_stop_details

# Fetch and append stop details
def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"
    try:
        for attempt in range(retries):
            response = requests.get(url)
            if response.status_code == 429:
                time.sleep(2**attempt)
                continue
            response.raise_for_status()
            data = response.json()
            stop_points = data.get('stopPoints', [])
            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')
                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })
                        printed_stop_ids.add(stop_id)
            else:
                continue
            break
        else:
            pass
    except requests.exceptions.RequestException as e:
        pass

# Write to CSV
def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']
    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()
        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)
    print(f"Data written to 'stop_details.csv' successfully.")

# Display data from CSV
def display_data_from_csv():
    print("Data from CSV:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        headers_printed = False
        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

# Get QSI stop points
def get_qsi_stop_points():
    count_of_qsi_stop_points = int(input("Enter the number of QSI stop points: "))
    qsi_stop_points = []
    for i in range(count_of_qsi_stop_points):
        row_no = int(input(f"Enter the row number for QSI stop point {i+1}: "))
        qsi_stop_points.append(row_no)
    qsi_stop_ids = []
    print("QSI Stop Points:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if int(row['Sl. No.']) in qsi_stop_points:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                qsi_stop_ids.append(row['Stop ID'])
    return qsi_stop_ids

# Main loop to fetch and display data
def main_loop(line_id, direction, qsi_stop_ids):
    while True:
        for idx, stop_point_id in enumerate(qsi_stop_ids):
            df, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)
            if df is not None and station_name is not None:
                clear_output(wait=True)
                total_wawt = df['WAWT'].sum()
                if not df.empty:
                    min_arrival = pd.to_datetime(df['Expected Arrival (BST)']).min().replace(second=0, microsecond=0)
                    max_arrival = pd.to_datetime(df['Expected Arrival (BST)']).max().replace(second=0, microsecond=0)
                    time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60
                    awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0
                    nbph = len(df) / (time_diff_minutes / 60)
                    swt = round(60 / (nbph * 2), 2)
                    ewt = round(awt - swt, 2)
                    summary_df = pd.DataFrame({
                        'Metric': ['Number of buses scheduled per hour (nbph)', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses (minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
                        'Value': [nbph, total_wawt, time_diff_minutes, awt, swt, ewt]
                    })
                    print(f"Arrival Predictions for QSI Stop Point {idx + 1} ({station_name}):")
                    display(df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']])
                    print("\nSummary Metrics:")
                    display(summary_df)
                else:
                    print(f"No data available for QSI Stop Point {idx + 1} ({station_name}).")
            else:
                print(f"Failed to fetch data for QSI Stop Point {idx + 1}.")
        time.sleep(15)

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    fetch_route_details(line_id, direction)
    qsi_stop_ids = get_qsi_stop_points()
    main_loop(line_id, direction, qsi_stop_ids)


Arrival Predictions for QSI Stop Point 1 (Moorfields Eye Hospital):


<ipython-input-2-d3051239c772>:194: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  min_arrival = pd.to_datetime(df['Expected Arrival (BST)']).min().replace(second=0, microsecond=0)
<ipython-input-2-d3051239c772>:195: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  max_arrival = pd.to_datetime(df['Expected Arrival (BST)']).max().replace(second=0, microsecond=0)


,Line,Vehicle ID,Stop Point,Direction,Expected Arrival (BST),Headway (minutes),AWT/bus (minutes),WAWT
1,214,LA19KBV,490010012N,inbound,10:07:00,0.0,0.0,0.0
0,214,LA19KBP,490010012N,inbound,10:14:00,7.0,3.5,24.5
2,214,LA19KBY,490010012N,inbound,10:22:00,8.0,4.0,32.0



Summary Metrics:


,Metric,Value
0,Number of buses scheduled per hour (nbph),12.00
1,Total WAWT (minutes),56.50
2,Time difference between 1st and last observed ...,15.00
3,AWT (minutes),3.77
4,SWT (minutes),2.50
5,EWT (minutes),1.27


Failed to fetch data for QSI Stop Point 2.
Failed to fetch data for QSI Stop Point 3.


KeyboardInterrupt: 

Only few stop point ids have next arrival data . Eg row nos: 25, 102, 125 in lineID 214 , inbound

In [4]:
# Step 2: Install required libraries (if not already installed)
!pip install requests pandas plotly dash

# Step 3: Import required libraries
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from IPython.display import display, clear_output
import plotly.express as px
import plotly.graph_objects as go
import pytz
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import csv

# Function to convert time to BST
def convert_to_bst(dt):
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')
    utc_dt = utc_tz.localize(dt)
    bst_dt = utc_dt.astimezone(bst_tz)
    return bst_dt.strftime('%H:%M:%S')

# Function to fetch arrival predictions with error handling
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None  # No data available
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

# Fetch route details
def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {'serviceTypes': 'Regular'}
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        route_sections = data.get('routeSections', [])
        found_direction = False
        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')
                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)
                if all_stop_details:
                    write_to_csv(all_stop_details)
                    display_data_from_csv()
        if not found_direction:
            print(f"No route found for direction '{direction}'.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

# Fetch and append stop points
def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        line_strings = data.get('lineStrings', [])
        if line_strings:
            printed_stop_ids = set()
            for line_string in line_strings:
                coordinates = eval(line_string)
                for lon, lat in coordinates[0]:
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)
        else:
            print("No stop points found for this route.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")
    return all_stop_details

# Fetch and append stop details
def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"
    try:
        for attempt in range(retries):
            response = requests.get(url)
            if response.status_code == 429:
                time.sleep(2**attempt)
                continue
            response.raise_for_status()
            data = response.json()
            stop_points = data.get('stopPoints', [])
            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')
                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })
                        printed_stop_ids.add(stop_id)
            else:
                continue
            break
        else:
            pass
    except requests.exceptions.RequestException as e:
        pass

# Write to CSV
def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']
    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()
        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)
    print(f"Data written to 'stop_details.csv' successfully.")

# Display data from CSV
def display_data_from_csv():
    print("Data from CSV:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        headers_printed = False
        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

# Get QSI stop points
def get_qsi_stop_points():
    count_of_qsi_stop_points = int(input("Enter the number of QSI stop points: "))
    qsi_stop_points = []
    for i in range(count_of_qsi_stop_points):
        row_no = int(input(f"Enter the row number for QSI stop point {i+1}: "))
        qsi_stop_points.append(row_no)
    qsi_stop_ids = []
    print("QSI Stop Points:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if int(row['Sl. No.']) in qsi_stop_points:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                qsi_stop_ids.append(row['Stop ID'])
    return qsi_stop_ids

# Main loop to fetch and display data
def main_loop(line_id, direction, qsi_stop_ids):
    while True:
        for idx, stop_point_id in enumerate(qsi_stop_ids):
            df, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)
            if df is not None and station_name is not None:
                clear_output(wait=True)
                total_wawt = df['WAWT'].sum()
                if not df.empty:
                    min_arrival = pd.to_datetime(df['Expected Arrival (BST)']).min().replace(second=0, microsecond=0)
                    max_arrival = pd.to_datetime(df['Expected Arrival (BST)']).max().replace(second=0, microsecond=0)
                    time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60
                    awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0
                    swt = round(60 / (nbph * 2), 2)
                    ewt = round(awt - swt, 2)
                    summary_df = pd.DataFrame({
                        'Metric': ['Number of buses scheduled per hour (nbph)', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses (minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
                        'Value': [nbph, total_wawt, time_diff_minutes, awt, swt, ewt]
                    })
                    print(f"Arrival Predictions for QSI Stop Point {idx + 1} ({station_name}):")
                    display(df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']])
                    print("\nSummary Metrics:")
                    display(summary_df)
                else:
                    print(f"No data available for QSI Stop Point {idx + 1} ({station_name}).")
            else:
                print(f"Failed to fetch data for QSI Stop Point {idx + 1}.")
        time.sleep(15)

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    fetch_route_details(line_id, direction)
    qsi_stop_ids = get_qsi_stop_points()
    main_loop(line_id, direction, qsi_stop_ids)


<ipython-input-4-bb08bdbea53f>:194: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  min_arrival = pd.to_datetime(df['Expected Arrival (BST)']).min().replace(second=0, microsecond=0)
<ipython-input-4-bb08bdbea53f>:195: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  max_arrival = pd.to_datetime(df['Expected Arrival (BST)']).max().replace(second=0, microsecond=0)


NameError: name 'nbph' is not defined

Taking nbph input from user

In [5]:
# Step 2: Install required libraries (if not already installed)
!pip install requests pandas plotly dash

# Step 3: Import required libraries
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from IPython.display import display, clear_output
import plotly.express as px
import plotly.graph_objects as go
import pytz
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import csv

# Function to convert time to BST
def convert_to_bst(dt):
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')
    utc_dt = utc_tz.localize(dt)
    bst_dt = utc_dt.astimezone(bst_tz)
    return bst_dt.strftime('%H:%M:%S')

# Function to fetch arrival predictions with error handling
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None  # No data available
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

# Fetch route details
def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {'serviceTypes': 'Regular'}
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        route_sections = data.get('routeSections', [])
        found_direction = False
        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')
                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)
                if all_stop_details:
                    write_to_csv(all_stop_details)
                    display_data_from_csv()
        if not found_direction:
            print(f"No route found for direction '{direction}'.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

# Fetch and append stop points
def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        line_strings = data.get('lineStrings', [])
        if line_strings:
            printed_stop_ids = set()
            for line_string in line_strings:
                coordinates = eval(line_string)
                for lon, lat in coordinates[0]:
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)
        else:
            print("No stop points found for this route.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")
    return all_stop_details

# Fetch and append stop details
def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"
    try:
        for attempt in range(retries):
            response = requests.get(url)
            if response.status_code == 429:
                time.sleep(2**attempt)
                continue
            response.raise_for_status()
            data = response.json()
            stop_points = data.get('stopPoints', [])
            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')
                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })
                        printed_stop_ids.add(stop_id)
            else:
                continue
            break
        else:
            pass
    except requests.exceptions.RequestException as e:
        pass

# Write to CSV
def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']
    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()
        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)
    print(f"Data written to 'stop_details.csv' successfully.")

# Display data from CSV
def display_data_from_csv():
    print("Data from CSV:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        headers_printed = False
        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

# Get QSI stop points
def get_qsi_stop_points():
    count_of_qsi_stop_points = int(input("Enter the number of QSI stop points: "))
    qsi_stop_points = []
    for i in range(count_of_qsi_stop_points):
        row_no = int(input(f"Enter the row number for QSI stop point {i+1}: "))
        qsi_stop_points.append(row_no)
    qsi_stop_ids = []
    print("QSI Stop Points:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if int(row['Sl. No.']) in qsi_stop_points:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                qsi_stop_ids.append(row['Stop ID'])
    return qsi_stop_ids

# Main loop to fetch and display data
def main_loop(line_id, direction, qsi_stop_ids):
    while True:
        for idx, stop_point_id in enumerate(qsi_stop_ids):
            df, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)
            if df is not None and station_name is not None:
                clear_output(wait=True)
                total_wawt = df['WAWT'].sum()
                min_arrival = pd.to_datetime(df['Expected Arrival (BST)'], format='%H:%M:%S').min().replace(second=0, microsecond=0)
                max_arrival = pd.to_datetime(df['Expected Arrival (BST)'], format='%H:%M:%S').max().replace(second=0, microsecond=0)
                time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60
                nbph = float(input(f"Enter the number of buses scheduled per hour for QSI Stop Point {idx + 1} ({station_name}): "))
                awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0
                swt = round(60 / (nbph * 2), 2)
                ewt = round(awt - swt, 2)
                summary_df = pd.DataFrame({
                    'Metric': ['Number of buses scheduled per hour (nbph)', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses (minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
                    'Value': [nbph, total_wawt, time_diff_minutes, awt, swt, ewt]
                })
                print(f"Arrival Predictions for QSI Stop Point {idx + 1} ({station_name}):")
                display(df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']])
                print("\nSummary Metrics:")
                display(summary_df)
            else:
                print(f"Failed to fetch data for QSI Stop Point {idx + 1}.")
        time.sleep(15)

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    fetch_route_details(line_id, direction)
    qsi_stop_ids = get_qsi_stop_points()
    main_loop(line_id, direction, qsi_stop_ids)


Enter the number of buses scheduled per hour for QSI Stop Point 1 (Moorfields Eye Hospital): 6
Arrival Predictions for QSI Stop Point 1 (Moorfields Eye Hospital):


,Line,Vehicle ID,Stop Point,Direction,Expected Arrival (BST),Headway (minutes),AWT/bus (minutes),WAWT
3,214,LA19KBY,490010012N,inbound,10:21:00,0.0,0.0,0.0
2,214,LA19KBJ,490010012N,inbound,10:30:00,9.0,4.5,40.5
1,214,LA19KAO,490010012N,inbound,10:37:00,7.0,3.5,24.5
0,214,LA19KAE,490010012N,inbound,10:44:00,7.0,3.5,24.5



Summary Metrics:


,Metric,Value
0,Number of buses scheduled per hour (nbph),6.00
1,Total WAWT (minutes),89.50
2,Time difference between 1st and last observed ...,23.00
3,AWT (minutes),3.89
4,SWT (minutes),5.00
5,EWT (minutes),-1.11


Failed to fetch data for QSI Stop Point 2.
Failed to fetch data for QSI Stop Point 3.


KeyboardInterrupt: 

In [6]:
# Step 2: Install required libraries (if not already installed)
!pip install requests pandas plotly dash

# Step 3: Import required libraries
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from IPython.display import display, clear_output
import plotly.express as px
import plotly.graph_objects as go
import pytz
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import csv

# Function to convert time to BST
def convert_to_bst(dt):
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')
    utc_dt = utc_tz.localize(dt)
    bst_dt = utc_dt.astimezone(bst_tz)
    return bst_dt.strftime('%H:%M:%S')

# Function to fetch arrival predictions with error handling
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None  # No data available
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

# Fetch route details
def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {'serviceTypes': 'Regular'}
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        route_sections = data.get('routeSections', [])
        found_direction = False
        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')
                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)
                if all_stop_details:
                    write_to_csv(all_stop_details)
                    display_data_from_csv()
        if not found_direction:
            print(f"No route found for direction '{direction}'.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

# Fetch and append stop points
def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        line_strings = data.get('lineStrings', [])
        if line_strings:
            printed_stop_ids = set()
            for line_string in line_strings:
                coordinates = eval(line_string)
                for lon, lat in coordinates[0]:
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)
        else:
            print("No stop points found for this route.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")
    return all_stop_details

# Fetch and append stop details
def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"
    try:
        for attempt in range(retries):
            response = requests.get(url)
            if response.status_code == 429:
                time.sleep(2**attempt)
                continue
            response.raise_for_status()
            data = response.json()
            stop_points = data.get('stopPoints', [])
            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')
                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })
                        printed_stop_ids.add(stop_id)
            else:
                continue
            break
        else:
            pass
    except requests.exceptions.RequestException as e:
        pass

# Write to CSV
def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']
    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()
        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)
    print(f"Data written to 'stop_details.csv' successfully.")

# Display data from CSV
def display_data_from_csv():
    print("Data from CSV:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        headers_printed = False
        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

# Get QSI stop points
def get_qsi_stop_points():
    count_of_qsi_stop_points = int(input("Enter the number of QSI stop points: "))
    qsi_stop_points = []
    for i in range(count_of_qsi_stop_points):
        row_no = int(input(f"Enter the row number for QSI stop point {i+1}: "))
        qsi_stop_points.append(row_no)
    qsi_stop_ids = []
    print("QSI Stop Points:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if int(row['Sl. No.']) in qsi_stop_points:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                qsi_stop_ids.append(row['Stop ID'])
    return qsi_stop_ids

# Main loop to fetch and display data
def main_loop(line_id, direction, qsi_stop_ids):
    nbph = 6  # Fixed number of buses per hour for all QSI stop points
    while True:
        for idx, stop_point_id in enumerate(qsi_stop_ids):
            df, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)
            if df is not None and station_name is not None:
                clear_output(wait=True)
                total_wawt = df['WAWT'].sum()
                min_arrival = pd.to_datetime(df['Expected Arrival (BST)'], format='%H:%M:%S').min().replace(second=0, microsecond=0)
                max_arrival = pd.to_datetime(df['Expected Arrival (BST)'], format='%H:%M:%S').max().replace(second=0, microsecond=0)
                time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60
                awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0
                swt = round(60 / (nbph * 2), 2)
                ewt = round(awt - swt, 2)
                summary_df = pd.DataFrame({
                    'Metric': ['Number of buses scheduled per hour (nbph)', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses (minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
                    'Value': [nbph, total_wawt, time_diff_minutes, awt, swt, ewt]
                })
                print(f"Arrival Predictions for QSI Stop Point {idx + 1} ({station_name}):")
                display(df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']])
                print("\nSummary Metrics:")
                display(summary_df)
            else:
                print(f"Failed to fetch data for QSI Stop Point {idx + 1}.")
        time.sleep(15)

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    fetch_route_details(line_id, direction)
    qsi_stop_ids = get_qsi_stop_points()
    main_loop(line_id, direction, qsi_stop_ids)


Arrival Predictions for QSI Stop Point 3 (Lady Somerset Road):


,Line,Vehicle ID,Stop Point,Direction,Expected Arrival (BST),Headway (minutes),AWT/bus (minutes),WAWT
2,214,LA19KBU,490008660N,inbound,10:37:00,0.0,0.0,0.0
3,214,LA19KBV,490008660N,inbound,10:44:00,7.0,3.5,24.5
1,214,LA19KBP,490008660N,inbound,10:52:00,8.0,4.0,32.0
4,214,LA19KBY,490008660N,inbound,11:00:00,8.0,4.0,32.0
0,214,LA19KBJ,490008660N,inbound,11:06:00,6.0,3.0,18.0



Summary Metrics:


,Metric,Value
0,Number of buses scheduled per hour (nbph),6.00
1,Total WAWT (minutes),106.50
2,Time difference between 1st and last observed ...,29.00
3,AWT (minutes),3.67
4,SWT (minutes),5.00
5,EWT (minutes),-1.33


KeyboardInterrupt: 

Working Code: Enter line ID, direction. Gives the origination and destination of the lineID. Writes the  stop points between origination and destination in the sequence as they appears in the given direction of travel. Asks the user to enter the no of QSI points and the row number corresponding to them from the csv file. Returns the arrival predictions and summary metrics of the given QSI points.

In [9]:
# Step 2: Install required libraries (if not already installed)
!pip install requests pandas plotly dash

# Step 3: Import required libraries
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from IPython.display import display, clear_output
import plotly.express as px
import plotly.graph_objects as go
import pytz
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import csv

# Function to convert time to BST
def convert_to_bst(dt):
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')
    utc_dt = utc_tz.localize(dt)
    bst_dt = utc_dt.astimezone(bst_tz)
    return bst_dt.strftime('%H:%M:%S')

# Function to fetch arrival predictions with error handling
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None  # No data available
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

# Fetch route details
def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {'serviceTypes': 'Regular'}
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        route_sections = data.get('routeSections', [])
        found_direction = False
        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')
                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)
                if all_stop_details:
                    write_to_csv(all_stop_details)
                    display_data_from_csv()
        if not found_direction:
            print(f"No route found for direction '{direction}'.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

# Fetch and append stop points
def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        line_strings = data.get('lineStrings', [])
        if line_strings:
            printed_stop_ids = set()
            for line_string in line_strings:
                coordinates = eval(line_string)
                for lon, lat in coordinates[0]:
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)
        else:
            print("No stop points found for this route.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")
    return all_stop_details

# Fetch and append stop details
def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"
    try:
        for attempt in range(retries):
            response = requests.get(url)
            if response.status_code == 429:
                time.sleep(2**attempt)
                continue
            response.raise_for_status()
            data = response.json()
            stop_points = data.get('stopPoints', [])
            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')
                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })
                        printed_stop_ids.add(stop_id)
            else:
                continue
            break
        else:
            pass
    except requests.exceptions.RequestException as e:
        pass

# Write to CSV
def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']
    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()
        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)
    print(f"Data written to 'stop_details.csv' successfully.")

# Display data from CSV
def display_data_from_csv():
    print("Data from CSV:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        headers_printed = False
        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

# Get QSI stop points
def get_qsi_stop_points():
    count_of_qsi_stop_points = int(input("Enter the number of QSI stop points: "))
    qsi_stop_points = []
    for i in range(count_of_qsi_stop_points):
        row_no = int(input(f"Enter the row number for QSI stop point {i+1}: "))
        qsi_stop_points.append(row_no)
    qsi_stop_ids = []
    print("QSI Stop Points:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if int(row['Sl. No.']) in qsi_stop_points:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                qsi_stop_ids.append(row['Stop ID'])
    return qsi_stop_ids

# Main loop to fetch and display data
def main_loop(line_id, direction, qsi_stop_ids):
    nbph = 6  # Fixed number of buses per hour for all QSI stop points
    while True:
        qsi_data = []  # List to accumulate data for all QSI stop points
        for idx, stop_point_id in enumerate(qsi_stop_ids):
            df, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)
            if df is not None and station_name is not None:
                total_wawt = df['WAWT'].sum()
                min_arrival = pd.to_datetime(df['Expected Arrival (BST)'], format='%H:%M:%S').min().replace(second=0, microsecond=0)
                max_arrival = pd.to_datetime(df['Expected Arrival (BST)'], format='%H:%M:%S').max().replace(second=0, microsecond=0)
                time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60
                awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0
                swt = round(60 / (nbph * 2), 2)
                ewt = round(awt - swt, 2)
                summary_df = pd.DataFrame({
                    'Metric': ['Number of buses scheduled per hour (nbph)', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses (minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
                    'Value': [nbph, total_wawt, time_diff_minutes, awt, swt, ewt]
                })
                qsi_data.append({
                    'QSI Stop Point': f"{idx + 1} - {station_name}",
                    'Arrival Predictions': df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']].to_string(index=False),
                    'Summary Metrics': summary_df.to_string(index=False)
                })
            else:
                print(f"Failed to fetch data for QSI Stop Point {idx + 1}.")

        clear_output(wait=True)
        for qsi in qsi_data:
            print(f"QSI Stop Point: {qsi['QSI Stop Point']}")
            print("Arrival Predictions:")
            print(qsi['Arrival Predictions'])
            print("\nSummary Metrics:")
            print(qsi['Summary Metrics'])
            print("\n--------------------------------------------\n")

        time.sleep(15)

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    fetch_route_details(line_id, direction)
    qsi_stop_ids = get_qsi_stop_points()
    main_loop(line_id, direction, qsi_stop_ids)


QSI Stop Point: 1 - Commondale
Arrival Predictions:
Line Vehicle ID Stop Point Direction Expected Arrival (BST)  Headway (minutes)  AWT/bus (minutes)  WAWT
  22    LF67EWP 490012375E  outbound               11:36:00                0.0                0.0   0.0
  22    BV66VHK 490012375E  outbound               11:47:00               11.0                5.5  60.5
  22    BV66VLN 490012375E  outbound               11:58:00               11.0                5.5  60.5

Summary Metrics:
                                                       Metric  Value
                    Number of buses scheduled per hour (nbph)    6.0
                                         Total WAWT (minutes)  121.0
Time difference between 1st and last observed buses (minutes)   22.0
                                                AWT (minutes)    5.5
                                                SWT (minutes)    5.0
                                                EWT (minutes)    0.5

------------------------------

KeyboardInterrupt: 

In [ ]:
# Step 2: Install required libraries (if not already installed)
!pip install requests pandas plotly dash

# Step 3: Import required libraries
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from IPython.display import display, clear_output
import plotly.express as px
import plotly.graph_objects as go
import pytz
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import csv

# Function to convert time to BST
def convert_to_bst(dt):
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')
    utc_dt = utc_tz.localize(dt)
    bst_dt = utc_dt.astimezone(bst_tz)
    return bst_dt.strftime('%H:%M:%S')

# Function to fetch arrival predictions with error handling
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None  # No data available
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

# Fetch route details
def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {'serviceTypes': 'Regular'}
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        route_sections = data.get('routeSections', [])
        found_direction = False
        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')
                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)
                if all_stop_details:
                    write_to_csv(all_stop_details)
                    display_data_from_csv()
        if not found_direction:
            print(f"No route found for direction '{direction}'.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

# Fetch and append stop points
def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        line_strings = data.get('lineStrings', [])
        if line_strings:
            printed_stop_ids = set()
            for line_string in line_strings:
                coordinates = eval(line_string)
                for lon, lat in coordinates[0]:
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)
        else:
            print("No stop points found for this route.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")
    return all_stop_details

# Fetch and append stop details
def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"
    try:
        for attempt in range(retries):
            response = requests.get(url)
            if response.status_code == 429:
                time.sleep(2**attempt)
                continue
            response.raise_for_status()
            data = response.json()
            stop_points = data.get('stopPoints', [])
            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')
                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })
                        printed_stop_ids.add(stop_id)
            else:
                continue
            break
        else:
            pass
    except requests.exceptions.RequestException as e:
        pass

# Write to CSV
def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']
    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()
        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)
    print(f"Data written to 'stop_details.csv' successfully.")

# Display data from CSV
def display_data_from_csv():
    print("Data from CSV:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        headers_printed = False
        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

# Get QSI stop points
def get_qsi_stop_points():
    count_of_qsi_stop_points = int(input("Enter the number of QSI stop points: "))
    qsi_stop_points = []
    for i in range(count_of_qsi_stop_points):
        row_no = int(input(f"Enter the row number for QSI stop point {i+1}: "))
        qsi_stop_points.append(row_no)
    qsi_stop_ids = []
    print("QSI Stop Points:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if int(row['Sl. No.']) in qsi_stop_points:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                qsi_stop_ids.append(row['Stop ID'])
    return qsi_stop_ids

# Main loop to fetch and display data
def main_loop(line_id, direction, qsi_stop_ids):
    nbph = 6  # Fixed number of buses per hour for all QSI stop points
    while True:
        qsi_data = []  # List to accumulate data for all QSI stop points
        for idx, stop_point_id in enumerate(qsi_stop_ids):
            df, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)
            if df is not None and station_name is not None:
                total_wawt = df['WAWT'].sum()
                min_arrival = pd.to_datetime(df['Expected Arrival (BST)'], format='%H:%M:%S').min().replace(second=0, microsecond=0)
                max_arrival = pd.to_datetime(df['Expected Arrival (BST)'], format='%H:%M:%S').max().replace(second=0, microsecond=0)
                time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60
                awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0
                swt = round(60 / (nbph * 2), 2)
                ewt = round(awt - swt, 2)
                summary_df = pd.DataFrame({
                    'Metric': ['Number of buses scheduled per hour (nbph)', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses (minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
                    'Value': [nbph, total_wawt, time_diff_minutes, awt, swt, ewt]
                })
                qsi_data.append({
                    'QSI Stop Point': f"{idx + 1} - {station_name}",
                    'Arrival Predictions': df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']].to_string(index=False),
                    'Summary Metrics': summary_df.to_string(index=False)
                })
            else:
                qsi_data.append({
                    'QSI Stop Point': f"{idx + 1} - Fetch Error",
                    'Arrival Predictions': "No data available",
                    'Summary Metrics': "No data available"
                })

        clear_output(wait=True)
        for qsi in qsi_data:
            print(f"QSI Stop Point: {qsi['QSI Stop Point']}")
            print("Arrival Predictions:")
            print(qsi['Arrival Predictions'])
            print("\nSummary Metrics:")
            print(qsi['Summary Metrics'])
            print("\n--------------------------------------------\n")

        time.sleep(15)

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    fetch_route_details(line_id, direction)
    qsi_stop_ids = get_qsi_stop_points()
    main_loop(line_id, direction, qsi_stop_ids)


QSI Stop Point: 1 - Commondale
Arrival Predictions:
Line Vehicle ID Stop Point Direction Expected Arrival (BST)  Headway (minutes)  AWT/bus (minutes)  WAWT
  22    BV66VHK 490012375E  outbound               11:47:00                0.0                0.0   0.0
  22    BV66VLN 490012375E  outbound               11:58:00               11.0                5.5  60.5
  22    BV66VJN 490012375E  outbound               12:10:00               12.0                6.0  72.0

Summary Metrics:
                                                       Metric  Value
                    Number of buses scheduled per hour (nbph)   6.00
                                         Total WAWT (minutes) 132.50
Time difference between 1st and last observed buses (minutes)  23.00
                                                AWT (minutes)   5.76
                                                SWT (minutes)   5.00
                                                EWT (minutes)   0.76

------------------------------